# LeNet5模型

In [9]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
from torch import optim
from torchvision import datasets, transforms

定义超参数

In [10]:
#parameters
epochs = 2
batch_size = 100
lr = 0.01#learn rate 

加载数据集

In [11]:
data_tf = transforms.Compose( [transforms.ToTensor(),transforms.Normalize([0.5], [0.5]), transforms.Resize(32, 32)])
train_data = torchvision.datasets.MNIST(
    root='./MNIST_data',
    train=True,
    transform = data_tf,
    download=True)
train_loader =Data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)

test_data = torchvision.datasets.MNIST(root='./MNIST_data',train=False,transform = data_tf,download=True)
test_loader =Data.DataLoader(dataset=test_data,batch_size=batch_size,shuffle=True)

test_x = Variable(torch.unsqueeze(test_data.test_data,dim=1),volatile=True).type(torch.FloatTensor)[:10000]/255.
test_y = test_data.test_labels[:10000]

print(train_data.train_data.size())                 
print(train_data.train_labels.size())   
print(test_data.test_data.size())                 
print(test_data.test_labels.size())  

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:12: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if sys.path[0] == '':


torch.Size([60000, 28, 28])
torch.Size([60000])
torch.Size([10000, 28, 28])
torch.Size([10000])


定义LeNet5模型，该模型有7层（不包括输入层），分别是：卷积层$\rightarrow 池化层 \rightarrow 卷积层 \rightarrow 池化层 \rightarrow 卷积层\rightarrow 全连接层 \rightarrow $输出层

In [16]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(    #(1,28,28)1是channel的维度，28*28为图片的长宽
                in_channels=1,#图片的层数，RGB=3,灰度=1
                out_channels=6,#filter的个数
                kernel_size=5,#filter的长宽
                stride=1,#每隔多少个移动
                #padding=2,#图片补0.if stride = 1,padding =(kernel_size-1)/2 = (5-1)/2 = 2
            ),   
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),#可以看成2*2的filter ,减小一半
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6,16,5,1),
            nn.ReLU(),
            nn.MaxPool2d(2)

        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(16,120,5,1),
            nn.ReLU(),
        )  
        self.fc = nn.Sequential(     
            nn.Linear(120,84),
            nn.Linear(84,10), # ten output
            nn.Softmax(dim=-1)
        )
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0),-1) 
        output = self.fc(x)
        return output
net = LeNet5()
print(net)# net architectuer

LeNet5(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Linear(in_features=84, out_features=10, bias=True)
    (2): Softmax()
  )
)


定义优化器和损失函数

In [13]:
opimizer = optim.Adam(net.parameters(),lr=lr)
loss_func = nn.CrossEntropyLoss()# loss function

训练数据

In [14]:
for epoch in range(epochs):
    for step,(x,y) in enumerate(train_loader):
        b_x = Variable(x)
        b_y = Variable(y)
        out = net(b_x)
        loss = loss_func(out,b_y)
        opimizer.zero_grad() # clear gradients for this training step
        loss.backward()# backpropagation, compute gradients
        opimizer.step()
        if step % 50 == 0:
            print('Epoch:',epoch,'|train loss:%.4f'% loss.item())

Epoch: 0 |train loss:2.2987
Epoch: 0 |train loss:0.3548
Epoch: 0 |train loss:0.2671
Epoch: 0 |train loss:0.1109
Epoch: 0 |train loss:0.1383
Epoch: 0 |train loss:0.0873
Epoch: 0 |train loss:0.0194
Epoch: 0 |train loss:0.1907
Epoch: 0 |train loss:0.0833
Epoch: 0 |train loss:0.0534
Epoch: 0 |train loss:0.0601
Epoch: 0 |train loss:0.0962
Epoch: 1 |train loss:0.3227
Epoch: 1 |train loss:0.0465
Epoch: 1 |train loss:0.0272
Epoch: 1 |train loss:0.2209
Epoch: 1 |train loss:0.0215
Epoch: 1 |train loss:0.0784
Epoch: 1 |train loss:0.0310
Epoch: 1 |train loss:0.0829
Epoch: 1 |train loss:0.0592
Epoch: 1 |train loss:0.0743
Epoch: 1 |train loss:0.0853
Epoch: 1 |train loss:0.0718


在测试集上测试

In [15]:
net.eval()
for step,(step_x,step_y) in enumerate(test_loader):
    test_out = net(test_x)
    pred_y = torch.max(test_out,1)[1].data.squeeze()
    correct = (pred_y == test_y).sum()
    #acc=float(correct)/test_y.size(0)
    acc = correct.item()/test_y.size(0)
print("acc:%.4f"% acc)#when epochs=100,acc:0.7615


KeyboardInterrupt

